In [216]:
import pickle
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle

In [2]:
def imputeDataNearest(sliceData, meanvalue):
    imputedata = []
    for i in range(len(sliceData)):
        a = np.array(sliceData[i])
        b = pd.DataFrame(np.array(a))
        b = b.replace(-1,np.nan)
        c = b.count()
        for k in range(a.shape[1]):
            if c[k] == 0:
                b[k] = meanvalue[k]
            if c[k] == 1:
                b[k] = b[k].interpolate(method ='ffill',axis=0)
                b[k] = b[k].interpolate(method ='bfill',axis=0)
        b = b.interpolate(method ='nearest',axis=0,limit_direction ='both')
        b = b.interpolate(method ='ffill')
        b = b.interpolate(method ='bfill')
        b = b.values
        imputedata.append(b)
    return imputedata

In [3]:
f = open('E:/WashU/Research/ICU/Data/train/X_train_sliced_norm.pkl','rb')
X_train_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/y_train.pkl','rb')
y_train = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_delta_mat.pkl','rb')
train_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_mask_mat.pkl','rb')
train_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/val/X_val_sliced_norm.pkl','rb')
X_val_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/y_val.pkl','rb')
y_val = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/val_delta_mat.pkl','rb')
val_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/val_mask_mat.pkl','rb')
val_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/test/X_test_sliced_norm.pkl','rb')
X_test_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/test/y_test.pkl','rb')
y_test = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/test/test_delta_mat.pkl','rb')
test_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/test/test_mask_mat.pkl','rb')
test_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/mean_norm.pkl','rb')
mean = pickle.load(f)
f.close()

In [146]:
X_train = np.array(imputeDataNearest(X_train_sliced, mean))
X_val = np.array(imputeDataNearest(X_val_sliced, mean))
train_delta_mat = np.array(train_delta_mat)
val_delta_mat = np.array(val_delta_mat)
train_mask_mat = np.array(train_mask_mat)
val_mask_mat = np.array(val_mask_mat)

In [73]:
class GRUICell(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs):
        super(GRUICell, self).__init__()
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
        def normal(shape):
            return torch.randn(size=shape) * 0.0001
        def three():
            return (nn.Parameter(normal((num_inputs, num_hiddens))), 
                    nn.Parameter(normal((num_hiddens, num_hiddens))), 
                    nn.Parameter(torch.zeros(num_hiddens)))
        self.W_xz, self.W_hz, self.b_z = three() # Parameters of update gate
        self.W_xr, self.W_hr, self.b_r = three() # Parameters of reset gate
        self.W_xh, self.W_hh, self.b_h = three() # Parameters of candidate hidden state
        # Parameters of decay vector
        self.W_beta = nn.Parameter(normal((num_inputs, num_hiddens)))
        self.b_beta = nn.Parameter(torch.zeros(num_hiddens))
        # Parameters of output layer
        self.W_hq = nn.Parameter(normal((num_hiddens, num_outputs)))
        self.b_q = nn.Parameter(torch.zeros(num_outputs))

    def forward(self, X, Delta, H):
        beta = torch.exp(torch.minimum(torch.zeros(self.num_hiddens), Delta @ self.W_beta + self.b_beta))
        H = beta * H
        Z = torch.sigmoid((X @ self.W_xz) + (H @ self.W_hz) + self.b_z)
        R = torch.sigmoid((X @ self.W_xr) + (H @ self.W_hr) + self.b_r)
        H_tilde = torch.tanh((X @ self.W_xh) + ((R * H) @ self.W_hh) + self.b_h)
        H = Z * H + (1 - Z) * H_tilde
        H.detach()
        Y = H @ self.W_hq + self.b_q
        return Y, H

In [74]:
class GRUIModel(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs):
        super(GRUIModel, self).__init__()
        self.name = 'GRUI'
        self.num_hiddens = num_hiddens
        self.gruicell = GRUICell(num_inputs, num_hiddens, num_outputs)

    def forward(self, X, Delta, H):
        if H is None:
            H = torch.zeros(X.shape[1], self.num_hiddens)
        Y = torch.tensor([])
        for index in range(X.shape[0]):
            Y_new, H = self.gruicell(X[index], Delta[index], H)
            Y = torch.cat((Y, Y_new), dim = 0)
        return Y, H

In [94]:
class PretrainGenerator(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs, drop_prob):
        super(PretrainGenerator, self).__init__()
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
        self.grui_model = GRUIModel(num_inputs, num_hiddens, num_outputs)
        self.drop_prob = drop_prob
        
        def normal(shape):
            return torch.randn(size=shape) * 0.0001
        
        # Parameters of fully connected layer
        self.W_fc = nn.Parameter(normal((num_outputs, num_inputs)))
        self.b_fc = nn.Parameter(torch.zeros(num_inputs))
    
    def forward(self, X, Delta, H):
        if H is None:
            H = torch.zeros(X.shape[1], self.num_hiddens)
        
        outputs, final_state = self.grui_model(X, Delta, H)
        outputs = outputs.view(X.shape[0], X.shape[1], self.num_outputs)
        # Full connect
        f_dropout = nn.Dropout(p = self.drop_prob)
        imputed_result = torch.tensor([])
        for i in range(outputs.shape[0]):
            out_predict = f_dropout(outputs[i]) @ self.W_fc + self.b_fc
            imputed_result = torch.cat((imputed_result, out_predict), dim = 0)
        imputed_result = imputed_result.view(X.shape[0], X.shape[1], self.num_inputs)
        
        return imputed_result

In [214]:
class Generator(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs, slice_gaps, z_dim, drop_prob):
        super(Generator, self).__init__()
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
        self.grui_model = GRUIModel(num_inputs, num_hiddens, num_outputs)
        self.slice_gaps = slice_gaps
        self.time_steps = 2880//slice_gaps
        self.z_dim = z_dim
        self.drop_prob = drop_prob
        
        def normal(shape):
            return torch.randn(size=shape) * 0.0001
        
        # Parameters of random vector to X
        self.W_rv2x = nn.Parameter(normal((z_dim, num_inputs)))
        self.b_x = nn.Parameter(torch.zeros(num_inputs))
        # Parameters of fully connected layer
        self.W_fc = nn.Parameter(normal((num_outputs, num_inputs)))
        self.b_fc = nn.Parameter(torch.zeros(num_inputs))
    
    def forward(self, rv, H, batch_size):
        if rv is None:
            rv = torch.randn((self.time_steps, batch_size, self.z_dim))
        X = torch.tensor([])
        for i in range(rv.shape[0]):
            X = torch.cat((X, rv[i] @ self.W_rv2x + self.b_x), dim = 0)
        X = X.view(self.time_steps, batch_size, self.num_inputs)
        if H is None:
            H = torch.zeros(batch_size, self.num_hiddens)
        Delta = self.slice_gaps*torch.ones(self.time_steps, batch_size, self.num_inputs)
        
        outputs, final_state = self.grui_model(X, Delta, H)
        outputs = outputs.view(self.time_steps, batch_size, self.num_outputs)
        # Full connect
        f_dropout = nn.Dropout(p = self.drop_prob)
        imputed_result = torch.tensor([])
        for i in range(outputs.shape[0]):
            out_predict = f_dropout(outputs[i]) @ self.W_fc + self.b_fc
            imputed_result = torch.cat((imputed_result, out_predict), dim = 0)
        imputed_result = imputed_result.view(self.time_steps, batch_size, self.num_inputs)
        
        # batch_normalization???
        
        return imputed_result, Delta, final_state

In [188]:
class Discriminator(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs, drop_prob):
        super(Discriminator, self).__init__()
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
        self.grui_model = GRUIModel(num_inputs, num_hiddens, num_outputs)
        self.drop_prob = drop_prob
        
        def normal(shape):
            return torch.randn(size=shape) * 0.0001

        # Parameters of fully connected layer
        self.W_fc = nn.Parameter(normal((num_outputs, 1)))
        self.b_fc = nn.Parameter(torch.zeros(1))
        
    def forward(self, X, Delta, H):
        if H is None:
            H = torch.zeros(X.shape[1], self.num_hiddens)
        outputs, final_state = self.grui_model(X, Delta, H)
        outputs = outputs.view(X.shape[0], X.shape[1], self.num_outputs)
        # Full connect
        f_dropout = nn.Dropout(p = self.drop_prob)
        real_logits = f_dropout(final_state) @ self.W_fc + self.b_fc
        real_probs = torch.sigmoid(real_logits)
        
        return real_probs, real_logits

In [241]:
class MRLoss(nn.Module):
    '''Masked Reconstruction Loss'''
    def __init__(self):
        super(MRLoss, self).__init__()
        pass
    
    def forward(self, X, M, X_g):
        mr_loss = 0
        for i in range(X.shape[0]):
            mr_loss += torch.norm(X[i] * M[i] - X_g[i] * M[i])
        mr_loss = mr_loss / X.shape[0]
        return mr_loss

class DLoss(nn.Module):
    '''Discriminative Loss'''
    def __init__(self):
        super(DLoss, self).__init__()
        pass
    
    def forward(self, D_real_logits, D_fake_logits):
        d_loss_real = - torch.mean(D_real_logits)
        d_loss_fake = torch.mean(D_fake_logits)
        d_loss = d_loss_real + d_loss_fake
        return d_loss

class GLoss(nn.Module):
    '''Loss for Generator'''
    def __init__(self):
        super(GLoss, self).__init__()
        pass
    
    def forward(self, D_fake_logits):
        g_loss = - torch.mean(D_fake_logits)
        return g_loss

class ImpLoss(nn.Module):
    '''Imputation Loss'''
    def __init__(self, g_loss_lambda):
        super(ImpLoss, self).__init__()
        self.g_loss_lambda = g_loss_lambda
    
    def forward(self, X, M, X_imp, imputed_fake_logits):
        imp_loss = 0
        for i in range(X.shape[0]):
            imp_loss += torch.norm(X[i] * M[i] - X_imp[:,i,:] * M[i]) - self.g_loss_lambda * imputed_fake_logits[i]
        imp_loss = imp_loss / X.shape[0]
        return imp_loss

In [121]:
def shuffle(x, y, z):
    # Turn into numpy
    if torch.is_tensor(x) == True:
        x = x.numpy()
    if torch.is_tensor(y) == True:
        y = y.numpy()
    if torch.is_tensor(z) == True:
        z = z.numpy()
    # Randomly exchange
    for i in range(len(x)):
        j = int(np.random.random() * (i + 1))
        if j <= len(x)-1:
            x[i], x[j] = x[j], x[i]
            y[i], y[j] = y[j], y[i]
            z[i], z[j] = z[j], z[i]

    # Turn back to tensor
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    z = torch.from_numpy(z)
    return x, y, z

In [115]:
def grad_clipping(net, theta):
    params = [p for p in net.parameters() if p.requires_grad]
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [206]:
def pretrain(model, X, M, Delta, batch_size, num_epoch, lr):
    criterion = MRLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr)
    for epoch in range(num_epoch):
        X, M, Delta = shuffle(X, M, Delta)
        for step in range(X.shape[0] // batch_size + 1):
            if (step + 1) * batch_size <= X_train.shape[0]:
                X_batch = X[int(step * batch_size):int((step + 1) * batch_size)]
                M_batch = M[int(step * batch_size):int((step + 1) * batch_size)]
                Delta_batch = Delta[int(step * batch_size):int((step + 1) * batch_size)]
            else:
                X_batch = X[int(step * batch_size):]
                M_batch = M[int(step * batch_size):]
                Delta_batch = Delta[int(step * batch_size):]
            outputs = model(X_batch.transpose(0, 1), Delta_batch.transpose(0, 1), None)
            X_batch_g = outputs.transpose(0, 1)
            loss = criterion(X_batch, M_batch, X_batch_g)
            optimizer.zero_grad()
            loss.backward()
            grad_clipping(model, 0.99)
            optimizer.step()
            # display training status
            print('Epoch: [%2d] [%4d/%4d] pretrain_loss: %.8f' % (epoch+1, step+1, X.shape[0] // batch_size + 1, loss))
    return model.state_dict()

In [218]:
def train(X, M, Delta, batch_size, num_epoch, num_pretrain_epoch, lr, disc_iters):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pretrain_model = PretrainGenerator(X.shape[2], 64, 64, 0.5).to(device)
    generator = Generator(X.shape[2], 64, 64, 60, 64, 0.5).to(device)
    discriminator = Discriminator(X.shape[2], 64, 64, 0.5).to(device)
    
    X = torch.from_numpy(X).float()
    M = torch.from_numpy(M).float()
    Delta = torch.from_numpy(Delta).float()
    
    pretrained_dict = pretrain(pretrain_model, X, M, Delta, batch_size, num_pretrain_epoch, 0.1)
    generator_dict = generator.state_dict()
    generator_dict.update(pretrained_dict)
    generator.load_state_dict(generator_dict)
    
    d_criterion = DLoss()
    g_criterion = GLoss()
    d_optimizer = torch.optim.SGD(discriminator.parameters(), lr)
    g_optimizer = torch.optim.SGD(generator.parameters(), lr * disc_iters)
    
    counter = 1
    
    for epoch in range(num_pretrain_epoch + 1, num_epoch+1):
        X, M, Delta = shuffle(X, M, Delta)
        for step in range(X.shape[0] // batch_size + 1):
            if (step + 1) * batch_size <= X_train.shape[0]:
                X_batch = X[int(step * batch_size):int((step + 1) * batch_size)]
                M_batch = M[int(step * batch_size):int((step + 1) * batch_size)]
                Delta_batch = Delta[int(step * batch_size):int((step + 1) * batch_size)]
            else:
                X_batch = X[int(step * batch_size):]
                M_batch = M[int(step * batch_size):]
                Delta_batch = Delta[int(step * batch_size):]
            
            # Update Discriminator Networks
            outputs, delta, final_state = generator(None, None, batch_size)
            d_real_probs, d_real_logits = discriminator(X_batch.transpose(0, 1), Delta_batch.transpose(0, 1), None)
            d_fake_probs, d_fake_logits = discriminator(outputs, delta, None)
            d_loss = d_criterion(d_real_logits, d_fake_logits)
            d_optimizer.zero_grad()
            d_loss.backward()
            # grad_clipping(discriminator, 1)
            d_optimizer.step()
            print('Epoch: [%2d] [%4d/%4d] d_loss: %.8f, counter: %4d'% (epoch, step+1, X.shape[0] // batch_size + 1, d_loss, counter))
            
            # Update Generator Networks
            if counter % disc_iters == 0:
                outputs, delta, final_state = generator(None, None, batch_size)
                d_real_probs, d_real_logits = discriminator(X_batch.transpose(0, 1), Delta_batch.transpose(0, 1), None)
                d_fake_probs, d_fake_logits = discriminator(outputs, delta, None)
                d_loss = d_criterion(d_real_logits, d_fake_logits)
                g_loss = g_criterion(d_fake_logits)
                g_optimizer.zero_grad()
                g_loss.backward()
                grad_clipping(generator, 0.99)
                g_optimizer.step()
                print('Epoch: [%2d] [%4d/%4d] d_loss: %.8f, g_loss: %.8f, counter: %4d'\
                      % (epoch, step+1, X.shape[0] // batch_size + 1, d_loss, g_loss, counter))
            
            counter += 1
    
    return generator, discriminator

In [258]:
generator, discriminator = train(X_val, val_mask_mat, val_delta_mat, 128, 30, 4, 0.01, 8)

Epoch: [ 1] [   1/  32] pretrain_loss: 10.99676418
Epoch: [ 1] [   2/  32] pretrain_loss: 10.55497646
Epoch: [ 1] [   3/  32] pretrain_loss: 9.92368317
Epoch: [ 1] [   4/  32] pretrain_loss: 9.41481686
Epoch: [ 1] [   5/  32] pretrain_loss: 9.02516747
Epoch: [ 1] [   6/  32] pretrain_loss: 8.34884262
Epoch: [ 1] [   7/  32] pretrain_loss: 8.20034218
Epoch: [ 1] [   8/  32] pretrain_loss: 7.75395966
Epoch: [ 1] [   9/  32] pretrain_loss: 7.22957993
Epoch: [ 1] [  10/  32] pretrain_loss: 6.45806026
Epoch: [ 1] [  11/  32] pretrain_loss: 6.42143250
Epoch: [ 1] [  12/  32] pretrain_loss: 5.76822138
Epoch: [ 1] [  13/  32] pretrain_loss: 6.08444691
Epoch: [ 1] [  14/  32] pretrain_loss: 5.29279661
Epoch: [ 1] [  15/  32] pretrain_loss: 5.23764181
Epoch: [ 1] [  16/  32] pretrain_loss: 5.33481598
Epoch: [ 1] [  17/  32] pretrain_loss: 4.21403503
Epoch: [ 1] [  18/  32] pretrain_loss: 3.80733418
Epoch: [ 1] [  19/  32] pretrain_loss: 4.47826529
Epoch: [ 1] [  20/  32] pretrain_loss: 3.9332578

Epoch: [ 5] [  28/  32] d_loss: -0.00000018, counter:   28
Epoch: [ 5] [  29/  32] d_loss: -0.00000020, counter:   29
Epoch: [ 5] [  30/  32] d_loss: -0.00000016, counter:   30
Epoch: [ 5] [  31/  32] d_loss: -0.00000022, counter:   31
Epoch: [ 5] [  32/  32] d_loss: -0.00000025, counter:   32
Epoch: [ 5] [  32/  32] d_loss: -0.00000022, g_loss: 0.00000021, counter:   32
Epoch: [ 6] [   1/  32] d_loss: -0.00000029, counter:   33
Epoch: [ 6] [   2/  32] d_loss: -0.00000024, counter:   34
Epoch: [ 6] [   3/  32] d_loss: -0.00000029, counter:   35
Epoch: [ 6] [   4/  32] d_loss: -0.00000029, counter:   36
Epoch: [ 6] [   5/  32] d_loss: -0.00000028, counter:   37
Epoch: [ 6] [   6/  32] d_loss: -0.00000030, counter:   38
Epoch: [ 6] [   7/  32] d_loss: -0.00000033, counter:   39
Epoch: [ 6] [   8/  32] d_loss: -0.00000036, counter:   40
Epoch: [ 6] [   8/  32] d_loss: -0.00000034, g_loss: 0.00000035, counter:   40
Epoch: [ 6] [   9/  32] d_loss: -0.00000039, counter:   41
Epoch: [ 6] [  1

Epoch: [ 9] [  19/  32] d_loss: -0.00000591, counter:  147
Epoch: [ 9] [  20/  32] d_loss: -0.00000609, counter:  148
Epoch: [ 9] [  21/  32] d_loss: -0.00000594, counter:  149
Epoch: [ 9] [  22/  32] d_loss: -0.00000648, counter:  150
Epoch: [ 9] [  23/  32] d_loss: -0.00000626, counter:  151
Epoch: [ 9] [  24/  32] d_loss: -0.00000698, counter:  152
Epoch: [ 9] [  24/  32] d_loss: -0.00000636, g_loss: 0.00000978, counter:  152
Epoch: [ 9] [  25/  32] d_loss: -0.00000692, counter:  153
Epoch: [ 9] [  26/  32] d_loss: -0.00000668, counter:  154
Epoch: [ 9] [  27/  32] d_loss: -0.00000725, counter:  155
Epoch: [ 9] [  28/  32] d_loss: -0.00000756, counter:  156
Epoch: [ 9] [  29/  32] d_loss: -0.00000755, counter:  157
Epoch: [ 9] [  30/  32] d_loss: -0.00000794, counter:  158
Epoch: [ 9] [  31/  32] d_loss: -0.00000827, counter:  159
Epoch: [ 9] [  32/  32] d_loss: -0.00000818, counter:  160
Epoch: [ 9] [  32/  32] d_loss: -0.00000787, g_loss: 0.00001196, counter:  160
Epoch: [10] [   

Epoch: [13] [  10/  32] d_loss: -0.00019336, counter:  266
Epoch: [13] [  11/  32] d_loss: -0.00020185, counter:  267
Epoch: [13] [  12/  32] d_loss: -0.00018675, counter:  268
Epoch: [13] [  13/  32] d_loss: -0.00020103, counter:  269
Epoch: [13] [  14/  32] d_loss: -0.00021437, counter:  270
Epoch: [13] [  15/  32] d_loss: -0.00022648, counter:  271
Epoch: [13] [  16/  32] d_loss: -0.00023444, counter:  272
Epoch: [13] [  16/  32] d_loss: -0.00023204, g_loss: 0.00040559, counter:  272
Epoch: [13] [  17/  32] d_loss: -0.00022968, counter:  273
Epoch: [13] [  18/  32] d_loss: -0.00027311, counter:  274
Epoch: [13] [  19/  32] d_loss: -0.00025795, counter:  275
Epoch: [13] [  20/  32] d_loss: -0.00030418, counter:  276
Epoch: [13] [  21/  32] d_loss: -0.00025566, counter:  277
Epoch: [13] [  22/  32] d_loss: -0.00028852, counter:  278
Epoch: [13] [  23/  32] d_loss: -0.00027676, counter:  279
Epoch: [13] [  24/  32] d_loss: -0.00028421, counter:  280
Epoch: [13] [  24/  32] d_loss: -0.0

Epoch: [17] [   1/  32] d_loss: -0.00744905, counter:  385
Epoch: [17] [   2/  32] d_loss: -0.00766375, counter:  386
Epoch: [17] [   3/  32] d_loss: -0.00725822, counter:  387
Epoch: [17] [   4/  32] d_loss: -0.00821858, counter:  388
Epoch: [17] [   5/  32] d_loss: -0.00813109, counter:  389
Epoch: [17] [   6/  32] d_loss: -0.00834346, counter:  390
Epoch: [17] [   7/  32] d_loss: -0.00852259, counter:  391
Epoch: [17] [   8/  32] d_loss: -0.00821674, counter:  392
Epoch: [17] [   8/  32] d_loss: -0.00829701, g_loss: 0.01783880, counter:  392
Epoch: [17] [   9/  32] d_loss: -0.00877223, counter:  393
Epoch: [17] [  10/  32] d_loss: -0.00892298, counter:  394
Epoch: [17] [  11/  32] d_loss: -0.01012378, counter:  395
Epoch: [17] [  12/  32] d_loss: -0.00907361, counter:  396
Epoch: [17] [  13/  32] d_loss: -0.00924245, counter:  397
Epoch: [17] [  14/  32] d_loss: -0.01074908, counter:  398
Epoch: [17] [  15/  32] d_loss: -0.01075248, counter:  399
Epoch: [17] [  16/  32] d_loss: -0.0

Epoch: [20] [  25/  32] d_loss: -0.11142541, counter:  505
Epoch: [20] [  26/  32] d_loss: -0.11760435, counter:  506
Epoch: [20] [  27/  32] d_loss: -0.10582697, counter:  507
Epoch: [20] [  28/  32] d_loss: -0.11768231, counter:  508
Epoch: [20] [  29/  32] d_loss: -0.13698235, counter:  509
Epoch: [20] [  30/  32] d_loss: -0.12604479, counter:  510
Epoch: [20] [  31/  32] d_loss: -0.13328987, counter:  511
Epoch: [20] [  32/  32] d_loss: -0.12470990, counter:  512
Epoch: [20] [  32/  32] d_loss: -0.12759089, g_loss: 0.31587911, counter:  512
Epoch: [21] [   1/  32] d_loss: -0.16327122, counter:  513
Epoch: [21] [   2/  32] d_loss: -0.14286219, counter:  514
Epoch: [21] [   3/  32] d_loss: -0.14405057, counter:  515
Epoch: [21] [   4/  32] d_loss: -0.16329956, counter:  516
Epoch: [21] [   5/  32] d_loss: -0.16043656, counter:  517
Epoch: [21] [   6/  32] d_loss: -0.17723440, counter:  518
Epoch: [21] [   7/  32] d_loss: -0.15676861, counter:  519
Epoch: [21] [   8/  32] d_loss: -0.1

KeyboardInterrupt: 

In [256]:
def imputation(generator, discriminator, X, M, z_dim, batch_size, lr, g_loss_lambda, impute_iters):
    batch_id = 1
    impute_times = 0
    counter = 0
    
    X = torch.from_numpy(X).float()
    M = torch.from_numpy(M).float()
    
    X_imputed = []
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for step in range(X.shape[0] // batch_size + 1):
        if (step + 1) * batch_size <= X_train.shape[0]:
            X_batch = X[int(step * batch_size):int((step + 1) * batch_size)]
            M_batch = M[int(step * batch_size):int((step + 1) * batch_size)]
            Delta_batch = Delta[int(step * batch_size):int((step + 1) * batch_size)]
        else:
            X_batch = X[int(step * batch_size):]
            M_batch = M[int(step * batch_size):]
            Delta_batch = Delta[int(step * batch_size):]
        
        generator_dict = generator.state_dict()
        gen = Generator(X.shape[2], 64, 64, 60, 64, 0).to(device)
        gen_dict = gen.state_dict()
        gen_dict.update(generator_dict)
        gen.load_state_dict(gen_dict)

        criterion = ImpLoss(g_loss_lambda)
        optimizer = torch.optim.SGD(gen.parameters(), lr)
        
        Z = torch.randn((X.shape[1], batch_size, z_dim)) * 0.1
        init_state = None
        for i in range(impute_iters):
            outputs, delta, final_state = gen(Z, init_state, batch_size)
            init_state = final_state.detach()
            imputed_fake_probs, imputed_fake_logits = discriminator(outputs, delta, None)
            loss = criterion(X_batch, M_batch, outputs, imputed_fake_logits)
            optimizer.zero_grad()
            loss.backward()
            grad_clipping(generator, 0.99)
            optimizer.step()
            impute_times += 1
            counter += 1
            if counter % 5 == 0:
                print("Batch ID: [%2d] [%2d/%2d] Imputation Loss: %.8f" % (batch_id, impute_times, impute_iters, loss))
        for j in range(X_batch.shape[0]):
            X_imputed.append((X_batch[j] * M_batch[j] + outputs[:,j,:] * (1 - M_batch[j])).detach().numpy())
        batch_id += 1
        impute_times = 0
    
    with open('E:/WashU/Research/ICU/Data/val/X_val_sliced_norm_GAN.pkl', 'wb') as f:
        pickle.dump(X_imputed, f)
        f.close()

In [257]:
imputation(generator, discriminator, X_val, val_mask_mat, 64, 128, 0.1, 5, 50)

Batch ID: [ 1] [ 5/50] Imputation Loss: 13.86453247
Batch ID: [ 1] [10/50] Imputation Loss: 6.51693058
Batch ID: [ 1] [15/50] Imputation Loss: 3.04967880
Batch ID: [ 1] [20/50] Imputation Loss: 1.82743025
Batch ID: [ 1] [25/50] Imputation Loss: 0.93843120
Batch ID: [ 1] [30/50] Imputation Loss: 0.48287624
Batch ID: [ 1] [35/50] Imputation Loss: 0.24892420
Batch ID: [ 1] [40/50] Imputation Loss: -0.67785907
Batch ID: [ 1] [45/50] Imputation Loss: -1.11262524
Batch ID: [ 1] [50/50] Imputation Loss: -2.33946323
Batch ID: [ 2] [ 5/50] Imputation Loss: 13.90427399
Batch ID: [ 2] [10/50] Imputation Loss: 6.31135130
Batch ID: [ 2] [15/50] Imputation Loss: 3.09396243
Batch ID: [ 2] [20/50] Imputation Loss: 1.32242930
Batch ID: [ 2] [25/50] Imputation Loss: 1.22556150
Batch ID: [ 2] [30/50] Imputation Loss: -0.50423497
Batch ID: [ 2] [35/50] Imputation Loss: -1.20657337
Batch ID: [ 2] [40/50] Imputation Loss: -0.80862945
Batch ID: [ 2] [45/50] Imputation Loss: -1.46083891
Batch ID: [ 2] [50/50]

Batch ID: [17] [ 5/50] Imputation Loss: 14.83633900
Batch ID: [17] [10/50] Imputation Loss: 6.69341326
Batch ID: [17] [15/50] Imputation Loss: 4.02208519
Batch ID: [17] [20/50] Imputation Loss: 1.99319696
Batch ID: [17] [25/50] Imputation Loss: 1.53960979
Batch ID: [17] [30/50] Imputation Loss: 0.21147019
Batch ID: [17] [35/50] Imputation Loss: 0.72578216
Batch ID: [17] [40/50] Imputation Loss: -0.89246786
Batch ID: [17] [45/50] Imputation Loss: -0.85776055
Batch ID: [17] [50/50] Imputation Loss: -0.93940181
Batch ID: [18] [ 5/50] Imputation Loss: 14.12462330
Batch ID: [18] [10/50] Imputation Loss: 6.39558125
Batch ID: [18] [15/50] Imputation Loss: 3.34516835
Batch ID: [18] [20/50] Imputation Loss: 2.32554054
Batch ID: [18] [25/50] Imputation Loss: 0.85587901
Batch ID: [18] [30/50] Imputation Loss: 0.65858197
Batch ID: [18] [35/50] Imputation Loss: -0.02751294
Batch ID: [18] [40/50] Imputation Loss: -1.31867886
Batch ID: [18] [45/50] Imputation Loss: -0.95203859
Batch ID: [18] [50/50] 

In [132]:
model.state_dict()

OrderedDict([('W_fc',
              tensor([[ 2.0291e-05, -2.0713e-05,  2.2791e-05,  ...,  3.2886e-05,
                       -1.4263e-05,  5.0891e-05],
                      [ 1.6621e-05,  1.0936e-05, -1.3899e-04,  ..., -4.6635e-05,
                       -5.6700e-05, -3.7246e-06],
                      [-1.7414e-04,  4.7840e-05,  4.9730e-05,  ..., -2.9710e-04,
                       -7.0029e-05,  1.4733e-04],
                      ...,
                      [ 1.5519e-04,  2.2104e-04, -7.2388e-05,  ..., -1.0975e-04,
                       -5.4459e-05,  1.0716e-04],
                      [-1.2450e-04,  1.6016e-04,  1.0652e-04,  ...,  5.1830e-06,
                       -2.0737e-05,  1.5034e-05],
                      [ 7.8309e-05,  5.5059e-05,  4.5348e-05,  ..., -9.2336e-05,
                        8.0715e-05,  3.3145e-05]])),
             ('b_fc',
              tensor([ 6.1708e-01, -2.8518e-05,  0.0000e+00,  1.1380e-04,  1.2502e-04,
                      -1.4223e-04,  1.0287e+00,  0.00

In [142]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PretrainGenerator(X_train.shape[2], 64, 64, 0.5)
pretrain(model, X_train, train_mask_mat, train_delta_mat, 128, 2, 0.1)

Epoch: [ 1] [   1/  32] pretrain_loss: 10.27311802
Epoch: [ 1] [   2/  32] pretrain_loss: 9.66712379
Epoch: [ 1] [   3/  32] pretrain_loss: 9.06840038
Epoch: [ 1] [   4/  32] pretrain_loss: 8.47798061
Epoch: [ 1] [   5/  32] pretrain_loss: 7.92711306
Epoch: [ 1] [   6/  32] pretrain_loss: 7.32745314
Epoch: [ 1] [   7/  32] pretrain_loss: 6.77063084
Epoch: [ 1] [   8/  32] pretrain_loss: 6.22888565
Epoch: [ 1] [   9/  32] pretrain_loss: 5.70473194
Epoch: [ 1] [  10/  32] pretrain_loss: 5.20105457
Epoch: [ 1] [  11/  32] pretrain_loss: 4.72102547
Epoch: [ 1] [  12/  32] pretrain_loss: 4.26782799
Epoch: [ 1] [  13/  32] pretrain_loss: 3.84420991
Epoch: [ 1] [  14/  32] pretrain_loss: 3.53320193
Epoch: [ 1] [  15/  32] pretrain_loss: 3.08936334
Epoch: [ 1] [  16/  32] pretrain_loss: 2.84161854
Epoch: [ 1] [  17/  32] pretrain_loss: 2.44299221
Epoch: [ 1] [  18/  32] pretrain_loss: 2.15234733
Epoch: [ 1] [  19/  32] pretrain_loss: 1.95744276
Epoch: [ 1] [  20/  32] pretrain_loss: 1.71622384

OrderedDict([('W_fc',
              tensor([[ 1.5201e-05, -2.1131e-05,  1.8640e-05,  ...,  2.8007e-05,
                        7.0239e-05, -5.8796e-05],
                      [ 7.3752e-06,  1.5036e-05,  5.2856e-05,  ..., -4.4628e-05,
                        2.4365e-04,  9.2550e-05],
                      [-2.4576e-04,  3.6707e-05,  6.5416e-05,  ..., -3.3574e-05,
                       -1.0504e-04,  1.0109e-04],
                      ...,
                      [-2.1998e-04,  4.0912e-05, -8.5859e-05,  ...,  1.8211e-05,
                       -1.0557e-04,  1.0375e-04],
                      [-1.0855e-04, -1.3562e-04, -2.3607e-05,  ...,  4.6210e-05,
                        9.9475e-05,  7.4551e-05],
                      [ 4.6493e-05, -2.5215e-05, -1.5866e-04,  ..., -6.6874e-05,
                       -4.0098e-05, -2.3900e-04]])),
             ('b_fc',
              tensor([ 6.0699e-01, -9.8817e-04,  2.6927e-03,  7.7233e-07,  3.6978e-02,
                       1.1600e-06,  9.6313e-01,  0.00

In [92]:
generator = Generator(X_resampled.shape[2], 64, 64, 60, 64, 0.5)
imputed_result, Delta = generator(None, None, 2)

discriminator = Discriminator(X_resampled.shape[2], 64, 64, 0.5)
real_prob = discriminator(imputed_result, Delta, None)
real_prob

tensor([[0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)

In [90]:
X_resampled.shape[2]

43

In [86]:
imputed_result.shape

torch.Size([48, 2, 43])

In [51]:
a = 30*torch.randn((3,2,4))
b = torch.tensor([])
for i in range(a.shape[0]):
    b = torch.cat((b,a[i]@torch.randn((4,5))),0)
b.view(3,2,5)

tensor([[[ 2.8283e+01, -1.0478e+02, -1.7709e+01,  8.2495e+01, -4.3884e+00],
         [-1.2151e+01,  1.3844e+01,  2.1678e+01,  9.0560e+00, -4.6305e+01]],

        [[-1.0245e+01,  1.3643e-01,  6.3071e+00,  8.2732e+00,  2.7988e+00],
         [ 3.1679e+01, -1.5437e+01, -2.6020e+01, -6.4427e+01, -4.0347e+01]],

        [[-1.3189e+01, -1.2936e+01,  4.2751e+01, -1.3718e+02, -7.9695e+01],
         [ 5.1605e+01,  7.9189e+01, -3.5383e+01,  3.8415e+01,  1.7778e+01]]])

In [102]:
a = torch.randn((3,2,4))
a

tensor([[[ 1.8225,  0.2152, -1.6502, -1.0723],
         [ 1.0081,  1.8549,  0.2984, -0.1364]],

        [[-0.7153, -0.3975,  0.1183,  1.3032],
         [-1.3603,  1.0030, -0.2714, -1.6327]],

        [[ 0.4245,  0.1874,  0.1894, -0.5705],
         [-1.3413,  0.0165, -0.9891, -0.6424]]])

In [108]:
a.transpose(0,1)

tensor([[[ 1.8225,  0.2152, -1.6502, -1.0723],
         [-0.7153, -0.3975,  0.1183,  1.3032],
         [ 0.4245,  0.1874,  0.1894, -0.5705]],

        [[ 1.0081,  1.8549,  0.2984, -0.1364],
         [-1.3603,  1.0030, -0.2714, -1.6327],
         [-1.3413,  0.0165, -0.9891, -0.6424]]])

In [69]:
b = torch.tensor([])
torch.cat((a,a),0).view(2,4,5)

tensor([[[ 0.1815, -0.5087,  0.1022, -0.0793,  1.6921],
         [-0.2830,  1.1934,  0.2826,  0.4353, -0.6111],
         [ 0.1957, -1.1001,  1.1122,  0.2993,  1.4969],
         [ 1.3750, -0.9169, -0.4234,  0.6335,  0.5345]],

        [[ 0.1815, -0.5087,  0.1022, -0.0793,  1.6921],
         [-0.2830,  1.1934,  0.2826,  0.4353, -0.6111],
         [ 0.1957, -1.1001,  1.1122,  0.2993,  1.4969],
         [ 1.3750, -0.9169, -0.4234,  0.6335,  0.5345]]])

In [70]:
m = nn.Dropout(p=0.2)
input = torch.randn(20, 16)
output = m(input)

In [110]:
int(np.random.random() * (10 + 1))

7